In [2]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch, helpers
import requests
from IPython.display import JSON
import numpy as np
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed
import pickle
from IPython.display import clear_output, display
from colavsim import colav_similarity, parse_doi, parse_string

In [3]:
es = Elasticsearch()

In [16]:
openalex = list(MongoClient()["openalexco"]["works"].find({'doi': {"$ne": None}},{"_id":0}))

In [17]:
len(openalex)

203863

In [18]:
es_index = "openalex_raw"
recreate = True
if recreate:
    if es.indices.exists(index=es_index):
        es.indices.delete(index=es_index)
if not es.indices.exists(index=es_index):
    es.indices.create(index=es_index)
if recreate:
    request_timeout = 60
    bulk_size = 100

    es_entries = []
    counter = 0
    for i in openalex:
        entry = {"_index": es_index,
                 "_id": counter,
                 "_source": i}
        es_entries.append(entry)
        if len(es_entries) == bulk_size:
            helpers.bulk(es, es_entries, refresh=True,request_timeout=request_timeout)
            es_entries = []
        counter +=1

In [ ]:
# settings = {
#   "settings": {
#     "index": {
#       "similarity": {
#         "default": {
#           "type": "DFR",
#           "basic_model": "g",
#           "after_effect": "b",
#           "normalization": "h2",
#           "normalization.h2.c": "600.0"  # Cambia el valor según tus necesidades
#         }
#       }
#     }
#   }
# }
# settings = {
#   "settings": {
#     "index": {
#       "similarity": {
#         "default": {
#           "type": "BM25",
#           "b": 0.80,   # Ajusta el parámetro "b" de BM25
#           "k1": 2.1   # Cambia el valor según tus necesidades
#         }
#       }
#     }
#   }
# }
settings={}

In [19]:
for i in range(len(openalex)):
    openalex[i]['title'] = parse_string(openalex[i]['title'])

In [20]:
es_index = "openalex_parsed"
recreate = True
if recreate:
    if es.indices.exists(index=es_index):
        es.indices.delete(index=es_index)
if not es.indices.exists(index=es_index):
    es.indices.create(index=es_index)

In [21]:
if recreate:
    request_timeout = 60
    bulk_size = 100

    es_entries = []
    counter = 0
    for i in openalex:
        entry = {"_index": es_index,
                 "_id": counter,
                 "_source": i}
        es_entries.append(entry)
        if len(es_entries) == bulk_size:
            helpers.bulk(es, es_entries, refresh=True,request_timeout=request_timeout)
            es_entries = []
        counter +=1

In [27]:
settings = {
  "settings": {
    "index": {
      "similarity": {
        "default": {
          "type": "DFR",
          "basic_model": "g",
          "after_effect": "b",
          "normalization": "h1",
          # "normalization.h2.c": "600.0"  # Cambia el valor según tus necesidades
        }
      }
    }
  }
}

In [28]:
es_index = "oa_dfr"
if not es.indices.exists(index=es_index):
    es.indices.create(index=es_index,body=settings)

/tmp/ipykernel_262558/3372967152.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=es_index,body=settings)


In [29]:
if recreate:
    request_timeout = 60
    bulk_size = 100

    es_entries = []
    counter = 0
    for i in openalex:
        entry = {"_index": es_index,
                 "_id": counter,
                 "_source": i}
        es_entries.append(entry)
        if len(es_entries) == bulk_size:
            helpers.bulk(es, es_entries, refresh=True,request_timeout=request_timeout)
            es_entries = []
        counter +=1

In [22]:
es.indices.delete(index="openalex_test")

{'acknowledged': True}